<a href="https://colab.research.google.com/github/DiDevv/Vector-Database/blob/main/recommender_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução:

Todo esse código estará armazenado no meu Github com o intuito tanto de guardar o meu aprendizado, quanto de compartilhar para outras pessoas.

Todo aprendizado aquiduirido aqui me foi proporcionado gratuitamente pela organização DeeplearningAI. O curso pode ser encontrado aqui: [Building Applications With Vector Databases](https://learn.deeplearning.ai/courses/building-applications-vector-databases)

A explicação do código estará em português e algumas abordagens alternativas foram tomadas pois estou rodando os códigos em um ambiente meu e não dentro do jupyter do próprio curso.

O curso ainda não possui legendas em português, foi um ponto de dificuldade para mim pois ainda estou aprimorando minha escuta do idioma inglês, por isso, tentarei abordar o conteúdo em português por aqui, para quebrar a barreira linguística para outras pessoas que ainda não conseguem "se virar" no inglês :P, eu te entendo!

Espero que possa ser útil, tanto pra você quanto pra mim!

# Recommender Syetems - Módulo 3

Sistemas de recomendação são utilizados em diversos setores da Indústria. Os bancos de dados vetoriais são capazes de otimizar a criação desses sistemas.

---

Antes de prosseguir, é necessário instalar alguns pacotes!

In [1]:
%pip install langchain openai pinecone tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 2.8 MB/s eta 0:00:00


# Importando Libs

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from tqdm.auto import tqdm, trange

import pandas as pd
import os
import time
import getpass

É necessário que você forneça tanto sua API do Pinecone, quanto a API da OpenAI, pois utilizaremos o modelo de Embedding deles!

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Forneça sua API da OpenAI")
os.environ["PINECONE_API_KEY"] = getpass.getpass("Forneça sua API da Pinecone")

Forneça sua API da OpenAI··········
Forneça sua API da Pinecone··········


In [4]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]

# Baixando o Dataset

Esse dataset foi disponibilizado pelo curso

In [5]:
!wget -q --show-progress -O all-the-news-3.zip "https://www.dropbox.com/scl/fi/wruzj2bwyg743d0jzd7ku/all-the-news-3.zip?rlkey=rgwtwpeznbdadpv3f01sznwxa&dl=1"

!unzip all-the-news-3.zip

all-the-news-3.zip  100%[===================>] 166.04M  87.0MB/s    in 1.9s    
Archive:  all-the-news-3.zip
  inflating: all-the-news-3.csv      


In [6]:
df = pd.read_csv('./all-the-news-3.csv', nrows=99) # Lendo as colunas do Dataset
df.head()

,date,year,month,day,author,title,article,url,section,publication
0,2016-12-09 18:31:00,2016,12.0,9,Lee Drutman,We should take concerns about the health of li...,"This post is part of Polyarchy, an independent...",https://www.vox.com/polyarchy/2016/12/9/138983...,NaN,Vox
1,2016-10-07 21:26:46,2016,10.0,7,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,NaN,Business Insider
2,2018-01-26 00:00:00,2018,1.0,26,NaN,Trump denies report he ordered Mueller fired,"DAVOS, Switzerland (Reuters) - U.S. President ...",https://www.reuters.com/article/us-davos-meeti...,Davos,Reuters
3,2019-06-27 00:00:00,2019,6.0,27,NaN,France's Sarkozy reveals his 'Passions' but in...,PARIS (Reuters) - Former French president Nico...,https://www.reuters.com/article/france-politic...,World News,Reuters
4,2016-01-27 00:00:00,2016,1.0,27,NaN,Paris Hilton: Woman In Black For Uncle Monty's...,Paris Hilton arrived at LAX Wednesday dressed ...,https://www.tmz.com/2016/01/27/paris-hilton-mo...,NaN,TMZ


In [57]:
df.columns

Index(['date', 'year', 'month', 'day', 'author', 'title', 'article', 'url',
       'section', 'publication'],
      dtype='object')

# Setup Pinecone

Com nosso dados baixados, bora montar um ambiente no Pinecone. Tratei disso no primeiro módulo do curso, você pode conferir [clicando aqui](https://github.com/DiDevv/Vector-Database/tree/main) e visitando o Similarity Search.

In [7]:
pinecone = Pinecone(api_key=PINECONE_API_KEY)
openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [8]:
index_name = "recommendation-systems"

pinecone.create_index(
    name=index_name,
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [9]:
index = pinecone.Index(index_name)

# Criando Embeddings de apenas os Títulos das Notícias

Um artigo muito interessante sobre esses modelos de embeddings está disponível no site da OpenAI.

[Clica aqui](https://platform.openai.com/docs/guides/embeddings) pra dar uma lida!

In [10]:
def get_embeddings(articles, model="text-embedding-ada-002"):
    return openai_client.embeddings.create(input = articles, model=model)

Enquanto eu estava estudando, confesso que só de bater o olho nesse "CHUNK" eu confundi com chunks que usamos em RAG (visto no módulo anterior).

Aqui esses chunks são diferentes, tamo garantindo que ao processar uma grande quantidade de addos com o pd, ele simplesmente não puxe toda nossa memória ram, pra isso, processamos em chunks, aos pouquinhos, pra garantir que tudo flua como esperado.

---

O resto do código é aquela velha gambiarra pra conseguirmos embeddar e enviar pro pinecone, nada de novo por aqui. Seguimos ^^

In [12]:
CHUNK_SIZE=400
TOTAL_ROWS=10000
progress_bar = tqdm(total=TOTAL_ROWS)
chunks = pd.read_csv('./all-the-news-3.csv', chunksize=CHUNK_SIZE,
                     nrows=TOTAL_ROWS)
chunk_num = 0
for chunk in chunks:
    titles = chunk['title'].tolist()
    embeddings = get_embeddings(titles)
    prepped = [{'id':str(chunk_num*CHUNK_SIZE+i), 'values':embeddings.data[i].embedding,
                'metadata':{'title':titles[i]},} for i in range(0,len(titles))]
    chunk_num = chunk_num + 1
    if len(prepped) >= 200:
      index.upsert(prepped)
      prepped = []
    progress_bar.update(len(chunk))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [13]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10000}},
 'total_vector_count': 10000}

# Sistema de Recomendação

Basicamente essa função trata de pegar lá no nosso pinecone_index o termo ao qual estamos pesquisando e retorna 10 buscas.

In [14]:
def get_recommendations(pinecone_index, search_term, top_k=10):
  embed = get_embeddings([search_term]).data[0].embedding
  res = pinecone_index.query(vector=embed, top_k=top_k, include_metadata=True)
  return res

Aqui eu vou loopar pela minha variável reco pra pegar todos os dados que foram retornados com o seu Score e o Metadado.

Pronto, temos nosso sistema de recomendação criado facilmente com um banco de dados vetorial.

In [23]:
reco = get_recommendations(index, 'beyonce')
for r in reco.matches:
    print(f'{r.score} : {r.metadata["title"]}')

0.852579176 : Super Bowl 50: Beyoncé Invites Coldplay, Bruno Mars, and the Football Players to Her Concert
0.851625204 : Beyonce Gives Epic Side-Eye to Wife of Warriors Owner
0.847580314 : The Zodiac Signs Ranked According to Beyoncé
0.845997036 : Beyonce: 'Formation' Concert POSTPONED!!
0.84172219 : Fan Rushes Stage At Beyonce, Jay-Z Concert in Atlanta
0.838265479 : Beyonce -- Cops Boycott ... Screw You Bey and Your Black Panthers
0.837614775 : A Beyoncé-Approved Virgo Explains How to Get Your Life Together
0.836674333 : Jay-Z & Beyonce Gussy Up for Rihanna's Charity Ball
0.832503498 : ​A Black Panther Halftime: The Revolutionary Politics Behind Beyoncé's Super Bowl Performance
0.828721046 : Beyonce & Jay Z: Screw Pineapples ... We're Pushing Lemons


In [25]:
reco = get_recommendations(index, 'adele')
for r in reco.matches:
    print(f'{r.score} : {r.metadata["title"]}')

0.820343435 : The Game Says Sorry Beyonce, Adele Deserved To Win
0.820249498 : Adele at the Grammys: “I can't possibly accept this award … my life is Beyoncé.”
0.812456727 : Adeline Gray is Wrestling For a Gold Medal and For Equality
0.809960306 : Watch Adele Freak Onstage Because a Bat Flew into Her Concert
0.799401045 : Madonna takes on frightening world with new album 'Madame X'
0.798507273 : Saying Goodbye to Aretha Franklin
0.797355354 : Ricotta Pansotti 
0.79449904 : Anders Is Secretly Ahead of Toronto's R&B World
0.793943226 : Vince Staples Joins James Blake on Stage in Glastonbury 
0.793590069 : Eddie Vedder and His 7-Year-Old Daughter Recorded a Cover of the 'Batman' Theme Song


# Criando Embeddings de todo o conteúdo das Notícias

Agora a gente vai fazer a mesma coisa, só que com o conteúdo das notícias.

---

Se você desejar criar um outro index para não perder o anterior, recomendo que crie um novo setup do Pinecone (vc já sabe como ^^)!

Nesse caso, como estou usando o plano gratuito do pinecone e ele só permite no máximo 5 indexes, irei apagar meu index anterior e criar um novo com o mesmo nome.

In [28]:
if index_name in [index.name for index in pinecone.list_indexes()]: # Esse código deleta nosso index anterior
  pinecone.delete_index(name=index_name)                            # E cria um novo com o mesmo nome

pinecone.create_index(name=index_name, dimension=1536, metric='cosine',
  spec=ServerlessSpec(cloud='aws', region='us-east-1'))
articles_index = pinecone.Index(index_name)

In [29]:
def embed(embeddings, title, prepped, embed_num):
  for embedding in embeddings.data:
    prepped.append({'id':str(embed_num), 'values':embedding.embedding, 'metadata':{'title':title}})
    embed_num += 1
    if len(prepped) >= 100:
        articles_index.upsert(prepped)
        prepped.clear()
  return embed_num

Lembra daquele conceito de Chunk que a gente tratou la em cima? É, sabia que ele apareceria em um momento ou outro e fiquei surpreso que ele apareceu ainda nesse módulo.

O text_splitter aqui serve para dividir os textos em chunks. Esses chunks servem para evitar extrapolar a quantidade de tokens que a gente pode enviar para um modelo de LLM (Se você não sabe o que são tokens, recomendo que pesquise sobre :) ). Existem várias técnicas de divisão desses chunks, mas a utilizada aqui é com base nesse algoritmo recursivo.

Aqui estamos limitando um chunk a no máximo 400 caracteres com uma sobreposição de 20 caracteres.
Isso ajuda a manter a integridade do texto.
Esses chunks são embeddados depois, e daí modelos podem buscar de forma assertiva os chunks que irão utilizar para responder o usuário evitando pegar os desnecessários que apenas serviriam para inflar o número de tokens de entrada. Deu pra entender? Espero que sim!

Um exemplo de divisão com um overlap de 20 caracteres:

Chunk 1:

```"O aprendizado de máquina é fascinante. Ele perm"```

Chunk 2 (sobreposição de 10 caracteres com Chunk 1):

```" fascinante. Ele permite que os computadores aprendam"```

Chunk 3 (sobreposição de 10 caracteres com Chunk 2):

```"omputadores aprendam com os dados e realizem tarefas complexas"```

In [31]:
news_data_rows_num = 500 # Quanto mais linhas você adicionar, mais lento será o processo
                         # Mas, melhor será o resultado.

embed_num = 0
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400,
    chunk_overlap=20)
prepped = []
df = pd.read_csv('./all-the-news-3.csv', nrows=news_data_rows_num)
articles_list = df['article'].tolist()
titles_list = df['title'].tolist()

for i in range(0, len(articles_list)):
    print(".",end="") # ajuda a acompanhar o progresso visualmente xD
    art = articles_list[i]
    title = titles_list[i]
    if art is not None and isinstance(art, str): #Aqui ele verifica que os artigos não são None e se são string
      texts = text_splitter.split_text(art)
      embeddings = get_embeddings(texts)
      embed_num = embed(embeddings, title, prepped, embed_num) # Se não entender o que está rolando aqui, dá uma olhada na função embed lá em cima ^^!

....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [32]:
articles_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5500}},
 'total_vector_count': 5500}

# Sistema de Recomendação

In [59]:
reco = get_recommendations(articles_index, 'Games', top_k=200)
seen = {}
for r in reco.matches:
    title = r.metadata['title']
    if title not in seen:
        print(f'{r.score} : {title}')
        seen[title] = '.'

0.81265229 : Tony Hawk Personally Picked Soundtrack for New Skate Video Game
0.811769426 : How much does lithium cost? The industry can't seem to agree
0.808714688 : Popular movies that were originally flops
0.80471617 : A Browser Game Recreates the Tedium of a 1990s Office Job
0.799053967 : Discussing the Enduring Appeal of Abe with Oddworld Creator Lorne Lanning
0.797410071 : The Virtual-Reality Future Is Here
0.797137201 : Fortnite meteors are falling: Here's what it means
0.794478595 : The Forgotten Artistic Playgrounds of the 20th Century
0.79368192 : A Painter's Fantasies Rendered as Cartoonish Dramas
0.791459799 : Take a Guess at What the Most Downloaded Kind of Porn Is in Anti-LGBT States
0.791047394 : It's 2012 All Over Again: We've Got 'Dragon's Dogma' Fever
0.78981936 : Liliana Porter Shows How Everything Familiar Must Be Magnified or Forgotten
0.789410055 : Kim says denuclearization “achievable” in second secret meeting with Xi 
0.787794948 : Finau fires first shot as 42nd 